In [1]:
from datetime import datetime, date, timedelta, time
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from quantrion import settings
from quantrion.ticker.alpaca import AlpacaTicker, AlpacaTickerListProvider
from quantrion.utils import MarketDatetime as mdt

load_dotenv()
%load_ext autoreload
%autoreload 2

In [2]:
ticker = AlpacaTicker("AAPL")

In [4]:
start = mdt.now() - timedelta(days=3)
df = await ticker.bars.get(start, freq="3min")
b_sma = await ticker.bars.get_sma(start, freq="3min", n=200)
lower, sma, upper = await ticker.bars.get_bollinger_bands(start, freq="3min")

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'https://data.alpaca.markets/v2/stocks/AAPL/bars?timeframe=1min&start=2022-08-28T09%3A15%3A00%2B00%3A00&end=2022-08-31T09%3A00%3A00%2B00%3A00'
For more information check: https://httpstatuses.com/422

In [6]:
from urllib.parse import urlparse

urlparse(
    "https://data.alpaca.markets/v2/stocks/AAPL/bars?timeframe=1min&start=2022-08-28T09%3A12%3A00%2B00%3A00&end=2022-08-31T08%3A57%3A00%2B00%3A00",
)

ParseResult(scheme='https', netloc='data.alpaca.markets', path='/v2/stocks/AAPL/bars', params='', query='timeframe=1min&start=2022-08-28T09%3A12%3A00%2B00%3A00&end=2022-08-31T08%3A57%3A00%2B00%3A00', fragment='')

In [ ]:
# 161.202512
# b_sma, bands
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Candlestick(
        x=df.index, open=df["open"], high=df["high"], low=df["low"], close=df["close"]
    ),
)
fig.add_trace(
    go.Scatter(x=sma.index, y=sma),
)
fig.add_trace(
    go.Scatter(x=lower.index, y=lower),
)
fig.add_trace(
    go.Scatter(x=upper.index, y=upper),
)
fig.add_trace(
    go.Scatter(
        x=b_sma.index,
        y=b_sma,
    ),
)
fig.update_xaxes(
    rangeslider_visible=True,
    rangebreaks=[
        dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
    ],
)
fig.show()

In [ ]:
# start = mdt.now() - timedelta(days=365)
# bars = await ticker.get_bars(start)
# bars = bars[(bars.index.time <= time(16, 0)) & (bars.index.time >= time(9, 30))]
# bars.to_csv("./files/apple_historical.csv")
# bars.head()

In [ ]:
_bars_resample_funcs = {
    "open": "first",
    "high": "max",
    "low": "min",
    "close": "last",
    "volume": "sum",
    "price": "sum",
    "n_trades": "sum",
}
bars = pd.read_csv("./files/apple_historical.csv")
bars["start"] = pd.DatetimeIndex(pd.to_datetime(bars["start"], utc=True)).tz_convert(
    "America/New_York"
)
bars = bars.set_index("start")
bars = bars[bars.index.dayofweek <= 4]
bars["price"] *= bars["volume"]
bars = bars.resample("3min").aggregate(_bars_resample_funcs).dropna()
bars["price"] /= bars["volume"]
_bars_fill_values = {
    "volume": 0,
}
bars = bars.fillna(_bars_fill_values)
na_index = bars["close"].isna()
bars["close"] = bars["close"].fillna(method="ffill")
for col in ["open", "high", "low"]:
    bars.loc[na_index, col] = bars["close"]
bars = bars.fillna(method="ffill")
bars.tail()

In [ ]:
spread = 0.012
tz = "America/New_York"
split_dt = pd.Timestamp("2022-06-01").tz_localize(tz)
bars_train = bars[:split_dt]
bars_test = bars[split_dt:]


def plot_candles(df):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
        ),
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df["SMA"]),
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df["LB"]),
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df["UB"]),
    )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["positions"],
        ),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["B_SMA"],
        ),
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
            dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
        ],
    )
    fig.update_yaxes(secondary_y=True)
    fig.show()


def plot_strategy(df):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_traces(
        [
            go.Scatter(x=df.index, y=df["strategy"], name="strategy"),
            go.Scatter(x=df.index, y=df["benchmark"], name="benchmark"),
            go.Scatter(x=df.index, y=df["trades"], name="trades"),
        ],
        secondary_ys=[False, False, True],
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
            dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
        ],
    )
    fig.show()


def process_bars(bars, sma_lag, big_sma_lag):
    bars = bars.copy()
    bars["SMA"] = bars["close"].rolling(sma_lag).mean()
    bars["B_SMA"] = bars["close"].rolling(big_sma_lag).mean()
    bars["STD"] = bars["close"].rolling(sma_lag).std()
    bars["LB"] = bars["SMA"] - 2 * bars["STD"]
    bars["UB"] = bars["SMA"] + 2 * bars["STD"]
    bars = bars.dropna(axis=0)
    bars["SMAdist"] = bars["B_SMA"] - bars["SMA"]
    bars.loc[
        (bars["close"] > bars["UB"]) & (bars["SMA"] < bars["B_SMA"]), "positions"
    ] = -1
    bars.loc[
        (bars["close"] < bars["LB"]) & (bars["SMA"] > bars["B_SMA"]), "positions"
    ] = 1
    bars.loc[bars["SMAdist"] * bars["SMAdist"].shift(1) < 0, "positions"] = 0

    #     taps_count = 0
    #     position = 0
    #     for index, bar in bars.iterrows():
    #         tap = bar["taps"]
    #         if tap == 0:
    #             continue
    #         if int(tap * taps_count) < 0:
    #             position = 0
    #             taps_count = 0
    #         taps_count += tap
    #         if tap < 0:
    #             position = 1
    #         if tap > 0:
    #             position = -1
    #         bars.loc[index, "positions"] = position
    #     bars["ctaps"] = bars.groupby(bars.index.date)["taps"].cumsum()

    #     bars.loc[bars[up_cross_stat] > bars["UB"], "positions"] = -1
    #     bars.loc[bars[low_cross_stat] < bars["LB"], "positions"] = 1
    #     bars.loc[bars.index.time >= time(15, 55), "positions"] = 0
    #     bars.loc[
    #         ((bars[low_cross_stat] <= bars["SMA"]) & (bars["positions"].ffill() == -1)) |
    #         ((bars[up_cross_stat] >= bars["SMA"]) & (bars["positions"].ffill() == 1)),
    #         "positions"
    #     ] = 0
    bars["positions"] = bars["positions"].ffill()
    bars["trades"] = bars["positions"].diff().abs().fillna(0).cumsum()
    bars["positions"] = bars["positions"].shift(1).fillna(0)
    bars["price_change"] = bars["close"].pct_change().fillna(0)
    bars["profit"] = bars["price_change"] * bars["positions"]
    bars["strategy"] = (1 + bars["profit"]).cumprod() - bars["trades"] * spread / bars[
        "price"
    ]
    bars["benchmark"] = (1 + bars["price_change"]).cumprod() - spread / bars["price"]
    return bars


import optuna


def objective(trial):
    sma_lag = trial.suggest_int("sma_lag", 10, 50)
    big_sma_lag = trial.suggest_int("big_sma_lag", sma_lag, 500)
    #     up_cross_stat = trial.suggest_categorical("up_cross_stat", ["open", "low", "high", "close", "price"])
    #     low_cross_stat = trial.suggest_categorical("low_cross_stat", ["open", "low", "high", "close", "price"])
    df = process_bars(bars_train, sma_lag, big_sma_lag)
    return (df["strategy"][-1] - df["strategy"][0]) / df["strategy"][0]


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

In [ ]:
study.trials_dataframe().sort_values("value", ascending=False).head(5)

In [ ]:
start = pd.Timestamp("2022-08-01T15:25:00").tz_localize(tz)
end = pd.Timestamp("2022-08-26T16:00:00").tz_localize(tz)
# df = process_bars(bars_train, 20, 403)
df = process_bars(bars_test, 13, 353)
# df = process_bars(bars_train, 13, 353)
plot_candles(df[start:end])
plot_strategy(df)


# plo(bars_test, 403, "low", "high", True)

In [ ]:
%timeit df.loc[:df.index[1]]